In [1]:
import torch
import torch.nn as nn

import torch.optim as optim
import torch.utils.data as data

from torch.utils.data import Dataset,DataLoader

import matplotlib.pyplot as plt
import numpy as np

import os
import random
import shutil
import time

import cv2

import json

from trd.utils import use_gpu


from seq_net import *
from data_tools import dataset_with_corners

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda:2")

In [3]:
folder = '/mnt/LAIN_EXTERNAL2/waterpolo_anyagok/vizilabda_meccsek/frames'
BATCH_SIZE = 8
EPOCH_NUM = 100
learning_rate = 0.001

In [4]:
train_json = json.load(open('train.json'))
val_json = json.load(open('val.json'))
test_json = json.load(open('test.json'))

In [5]:
train_ds = dataset_with_corners(folder,train_json)
val_ds = dataset_with_corners(folder,val_json)
test_ds = dataset_with_corners(folder,test_json)

train_iterator = data.DataLoader(train_ds, 
                                 batch_size = BATCH_SIZE)
val_iterator = data.DataLoader(val_ds, 
                               batch_size = BATCH_SIZE)
test_iterator = data.DataLoader(test_ds, 
                                batch_size = BATCH_SIZE)

In [6]:

criterion = nn.SmoothL1Loss().to(device)
model = seq_model
optimizer = optim.Adagrad(model.parameters(),lr=0.0005)

In [ ]:
val_loss = 1000 
for epoch in range(EPOCH_NUM): 

#set the running loss at each epoch to zero
    running_loss = 0.0
# we will enumerate the train loader with starting index of 0
# for each iteration (i) and the data (tuple of input and labels)
    for i, data in enumerate(train_iterator, 0):
        inputs, labels,fnames = data[0].cuda(), data[1].cuda(),data[2]
        #print(labels)
        # clear the gradient
        #inputs, labels,fnames = data[0], data[1],data[2]
        optimizer.zero_grad()
        #feed the input and acquire the output from network
        #outputs = net(inputs)
        #outputs = torch.stack(list(net(inputs)), dim=0)
        #outputs = outputs.to(torch.float32)
        outputs = model(inputs)
        #print(outputs)
        #outputs = outputs.reshape(-1)
        #calculating the predicted and the expected loss
        
        loss = criterion(outputs, labels)

        #compute the gradient
        #torch.cuda.empty_cache()
        loss.backward()

        #update the parameters
        optimizer.step()

        # print statistics
        #print(loss.item())
        running_loss += loss.item()
        prev_loss = 1
        if i % 200 == 0:
            print('[%d, %5d] loss: %.7f' % (epoch + 1, i + 1, running_loss/200))
            
            img = cv2.imread(os.path.join(folder,fnames[7]))
            print((outputs[7][0].item(),outputs[7][1].item()),(outputs[7][2].item(),outputs[7][3].item()))
            print((labels[7][0].item(),labels[7][1].item()),(labels[7][2].item(),labels[7][3].item()))
            print((round(outputs[7][0].item()*img.shape[1]),round(outputs[7][1].item()*img.shape[0])),(round(outputs[7][2].item()*img.shape[1]),round(outputs[7][3].item()*img.shape[0])))#(outputs[7][0].item(),outputs[7][1].item()),(outputs[7][2].item(),outputs[7][3].item()))
            #print((round(labels[7][0].item()*img.shape[1]),round(labels[7][1].item()*img.shape[0])),(round(labels[7][2].item()*img.shape[1]),round(labels[7][3].item()*img.shape[0])))#(labels[7][0].item(),labels[7][1].item()),(labels[7][2].item(),labels[7][3].item()))
            #im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            #im_rgb = cv2.rectangle(im_rgb,(round(outputs[7][0].item()*img.shape[1]),round(outputs[7][1].item()*img.shape[0])),(round(outputs[7][2].item()*img.shape[1]),round(outputs[7][3].item()*img.shape[0])),(255,0,0),2)
            #im_rgb = cv2.rectangle(im_rgb,(round(labels[7][0].item()*img.shape[1]),round(labels[7][1].item()*img.shape[0])),(round(labels[7][2].item()*img.shape[1]),round(labels[7][3].item()*img.shape[0])),(0,0,255),2)
            #plt.figure(figsize = (20,15))
            #plt.imshow(im_rgb)
            
            #torch.save(net.state_dict(),"net_epoch"+str(epoch)+'_'+str(i)+'_loss='+str(running_loss)+'.pt')
                
            running_loss = 0.0
            
        
        
    for i, data in enumerate(val_iterator, 0):
        inputs, labels,fnames = data[0].cuda(), data[1].cuda(),data[2]
        #print(labels)
        # clear the gradient
        #inputs, labels,fnames = data[0], data[1],data[2]
        optimizer.zero_grad()
        #feed the input and acquire the output from network
        #outputs = net(inputs)
        #outputs = torch.stack(list(net(inputs)), dim=0)
        #outputs = outputs.to(torch.float32)
        outputs = model(inputs)
        #print(outputs)
        #outputs = outputs.reshape(-1)
        #calculating the predicted and the expected loss

        loss = criterion(outputs, labels)

        # print statistics
        #print(loss.item())
        running_loss += loss.item()

    print('validation loss: %.7f' % (running_loss / i))

    if running_loss/i < val_loss:
        print('SAVE')
        torch.save(model.state_dict(),"flattened_model_net_epoch"+str(epoch)+'_loss='+str(running_loss/i)+'.pt')
        val_loss = running_loss/i

/home/csenge/.local/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,     1] loss: 0.0012330
(0.005841678008437157, 0.05452651530504227) (0.013849986717104912, -0.0011330305133014917)
(0.8494791388511658, 0.5138888955116272) (0.8755208253860474, 0.5509259104728699)
(11, 59) (27, -1)
[1,   201] loss: 0.0512151
(0.45400384068489075, 0.3892158567905426) (0.5590235590934753, 0.4291027784347534)
(0.39375001192092896, 0.44907405972480774) (0.41614583134651184, 0.49166667461395264)
(872, 420) (1073, 463)
[1,   401] loss: 0.0057448
(0.7544519305229187, 0.4328368008136749) (0.7711896896362305, 0.47489356994628906)
(0.7437499761581421, 0.40648147463798523) (0.7671874761581421, 0.43703705072402954)
(1449, 467) (1481, 513)
validation loss: 0.0314307
SAVE
[2,     1] loss: 0.0000458
(0.6932628154754639, 0.4162004888057709) (0.7114377021789551, 0.45320814847946167)
(0.8494791388511658, 0.5138888955116272) (0.8755208253860474, 0.5509259104728699)
(1331, 449) (1366, 489)
[2,   201] loss: 0.0058665
(0.5753222703933716, 0.3814900517463684) (0.5989092588424683, 0.421655

In [2]:

for i, data in enumerate(test_iterator, 0):
    if i < 20 :
        inputs, labels,fnames = data[0].cuda(), data[1].cuda(),data[2]
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        print('[%d] loss: %.7f' % ( i + 1, loss))
        outputs = outputs.to(torch.float32)
        img = cv2.imread(os.path.join(folder,fnames[7]))
        print(img.shape)

        print((outputs[7][0].item(),outputs[7][1].item()),(outputs[7][2].item(),outputs[7][3].item()))
        print((labels[7][0].item(),labels[7][1].item()),(labels[7][2].item(),labels[7][3].item()))
        #print((round(outputs[7][0].item()*img.shape[1]),round(outputs[7][1].item()*img.shape[0])),(round(outputs[7][2].item()*img.shape[1]),round(outputs[7][3].item()*img.shape[0])))#(outputs[7][0].item(),outputs[7][1].item()),(outputs[7][2].item(),outputs[7][3].item()))
        #print((round(labels[7][0].item()*img.shape[1]),round(labels[7][1].item()*img.shape[0])),(round(labels[7][2].item()*img.shape[1]),round(labels[7][3].item()*img.shape[0])))#(labels[7][0].item(),labels[7][1].item()),(labels[7][2].item(),labels[7][3].item()))
        im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        im_rgb = cv2.rectangle(im_rgb,(round(outputs[7][0].item()*img.shape[1]),round(outputs[7][1].item()*img.shape[0])),(round(outputs[7][2].item()*img.shape[1]),round(outputs[7][3].item()*img.shape[0])),(255,0,0),2)
        im_rgb = cv2.rectangle(im_rgb,(round(labels[7][0].item()*img.shape[1]),round(labels[7][1].item()*img.shape[0])),(round(labels[7][2].item()*img.shape[1]),round(labels[7][3].item()*img.shape[0])),(0,0,255),2)
        plt.figure(figsize = (20,15))
        plt.imshow(im_rgb)
        cv2.imwrite('testcase_{0}_coordinates.png'.format(i),im_rgb)
    else:
        break

NameError: name 'test_iterator' is not defined